# 계수기와 Timer 콘트랙

In [1]:
!geth --exec eth.accounts attach http://localhost:8445

["0x9b0b6b0b3bbdbd746427c7c3afe05282b52f08dd", "0xc78c8063043994e244384ff80b4153772b5d5cd2", "0x13516d2b36eb50a7300e2bab450f75d39f8422f3"]


In [4]:
!node src/getMyAddr.js

[
  '0x3534A45129b0776F1E811Ad51A74D46ebCBE6108',
  '0x1BE21dB844d547f2FCeFd0A93F0b43fAc6c068fC',
  '0x5Ed0D349E57aB3Fe8B1c564CFCbbFD3799E0Fd05',
  '0xC56f3F44e4Cd030Abb7B99200A3362324762a15A',
  '0x6D546D9aA842232FFE87Ea3D14526B1643FD09a1',
  '0xBF3f3C915d21E84cb058eD3A48Bee75866521743',
  '0xAA1DeBdb2909f124569B63e6a4372B72f9897E7B',
  '0x1186108091719CEFE4aB740EBE85eE993e8E50d8',
  '0xe611a20D34B9041E6A3704482FEb89FB5D722850',
  '0xD3B41D3884AAC2b8B6df1b2f24DcB51d194b1063'
]


### 1. 개발

In [2]:
%%writefile src/counterNtimer.sol
pragma solidity ^0.6.0;

contract counterNtimer {
    uint256 startTime;
    uint256 counter = 0;
    
    function start() public {
        startTime = now;
    }
    
    function timePassed() public view returns(uint256) {
        return now-startTime;
    }
    
    function getNow() public view returns(uint) {
        return now;
    }
    
    function add() public {
        counter++;
    }
    
    function subtract() public {
        counter--;
    }
    
    function getCounter() public view returns(uint256) {
        return counter;
    }
}

Writing src/counterNtimer.sol


### 2. 컴파일

In [3]:
!solc --abi --bin --gas src/counterNtimer.sol


======= src/counterNtimer.sol:counterNtimer =======
Gas estimation:
construction:
   5129 + 71200 = 76329
external:
   add():	20959
   getCounter():	1035
   getNow():	278
   start():	20245
   subtract():	20984
   timePassed():	1062
Binary:
6080604052600060015534801561001557600080fd5b50610164806100256000396000f3fe608060405234801561001057600080fd5b50600436106100625760003560e01c80634f2be91f146100675780636deebae3146100715780638ada066e1461007b578063b445425314610099578063bbe4fd50146100b7578063be9a6555146100d5575b600080fd5b61006f6100df565b005b6100796100f3565b005b610083610108565b6040518082815260200191505060405180910390f35b6100a1610112565b6040518082815260200191505060405180910390f35b6100bf61011d565b6040518082815260200191505060405180910390f35b6100dd610125565b005b600160008154809291906001019190505550565b60016000815480929190600190039190505550565b6000600154905090565b600080544203905090565b600042905090565b4260008190555056fea264697066735822122096f7be6e9d1ebffa231115b6471029a7a4fb230f178ec7a89e7d8ffd81a

### 3. 컨트랙 배포

In [7]:
%%writefile src/counterNtimerDeploy.js
var Web3 = require('web3');
var web3;

if(typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
}
else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

var _abiArray = [{"constant":false,"inputs":[],"name":"add","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"getCounter","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getNow","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"start","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"subtract","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"timePassed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}];
var _bin = "6080604052600060015534801561001557600080fd5b50610164806100256000396000f3fe608060405234801561001057600080fd5b50600436106100625760003560e01c80634f2be91f146100675780636deebae3146100715780638ada066e1461007b578063b445425314610099578063bbe4fd50146100b7578063be9a6555146100d5575b600080fd5b61006f6100df565b005b6100796100f3565b005b610083610108565b6040518082815260200191505060405180910390f35b6100a1610112565b6040518082815260200191505060405180910390f35b6100bf61011d565b6040518082815260200191505060405180910390f35b6100dd610125565b005b600160008154809291906001019190505550565b60016000815480929190600190039190505550565b6000600154905090565b600080544203905090565b600042905090565b4260008190555056fea264697066735822122096f7be6e9d1ebffa231115b6471029a7a4fb230f178ec7a89e7d8ffd81a76dbd64736f6c63430006040033";
var _contract = new web3.eth.Contract(_abiArray);

_contract
    .deploy({data:"0x"+_bin})
    .send({from:"0x3534A45129b0776F1E811Ad51A74D46ebCBE6108", gas:364124, gasPrice:'1000000000'})
    .then(function(newContractInstance) {
        console.log(newContractInstance.options.address)
    });

Overwriting src/counterNtimerDeploy.js


In [9]:
!node src/counterNtimerDeploy.js

0x2C29680D75b8377bf96Fb87a683b5dD1FA8353eB


### 4. 사용

In [23]:
%%writefile src/counterNtimerUse.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var abi = [{"constant":true,"inputs":[],"name":"add","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"getCounter","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getNow","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"start","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"subtract","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"timePassed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}];
var addr = "0x2C29680D75b8377bf96Fb87a683b5dD1FA8353eB";
var counter = new web3.eth.Contract(abi, addr);

// counter.methods.getCounter().call().then(function(str) {console.log(str);});
counter.methods.start().send({from:"0x3534A45129b0776F1E811Ad51A74D46ebCBE6108", gas:100000});
counter.methods.start().send({from:"0x3534A45129b0776F1E811Ad51A74D46ebCBE6108", gas:100000});
counter.methods.start().send({from:"0x3534A45129b0776F1E811Ad51A74D46ebCBE6108", gas:100000});
counter.methods.start().send({from:"0x3534A45129b0776F1E811Ad51A74D46ebCBE6108", gas:100000});
counter.methods.getCounter().call().then(function(str) {console.log("counter : ", str)});
counter.methods.timePassed().call().then(function(value) {console.log("timePassed : ", value)});

Overwriting src/counterNtimerUse.js


In [22]:
!node src/counterNtimerUse.js

0
counter :  0
timePassed :  0


In [37]:
!node src/counterNtimerUse.js

counter :  0
timePassed :  0


In [25]:
!node src/counterNtimerUse.js

counter :  0
timePassed :  0


In [36]:
!node src/counterNtimerUse.js

counter :  0
timePassed :  0


In [11]:
!node src/counterNtimerUse.js

counter 0
timePassed :  1650701592


In [11]:
!node src/counterNtimerUse.js

counter 0
timePassed :  1650701592


In [29]:
!solc --optimize --combined-json abi,bin src/counterNtimer.sol > src/counterNtimer.json

In [32]:
!type src\counterNtimer.json

{"contracts":{"src/counterNtimer.sol:counterNtimer":{"abi":"[{\"inputs\":[],\"name\":\"add\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getCounter\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"getNow\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"start\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"subtract\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"timePassed\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"}]","bin":"6080604052600060015534801561001557600080fd5b5060fb806100246000396000f3fe6080604052348015600f5760008

In [33]:
%%writefile src/counterNtimerImportTest.js
const _abiBinJson = require('./counterNtimer.json');
contractName = Object.keys(_abiBinJson.contracts);
console.log("- conatract name : ", contractName);
_abi = _abiBinJson.contracts[contractName].abi;
_abiArray = JSON.parse(JSON.stringify(_abi));
_bin = _abiBinJson.contracts[contractName].bin;
console.log("- abi : ", _abi);
console.log("- abi array : ", _abiArray);
console.log("- bytecode : ", _bin);

Writing src/counterNtimerImportTest.js


In [35]:
!node src/counterNtimerImportTest.js

- conatract name :  [ 'src/counterNtimer.sol:counterNtimer' ]
- abi :  [{"inputs":[],"name":"add","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"getCounter","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getNow","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"start","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"subtract","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"timePassed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]
- abi array :  [{"inputs":[],"name":"add","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"getCounter","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view"